# Text 1: Vector space models
**Internet Analytics - Lab 4**

---

**Group:** *J*

**Names:**

* *Kenza Driss*
* *Maximilien Hoffbeck*
* *Jaeyi Jeong*
* *Yoojin Kim*

---

#### Instructions

*This is a template for part 1 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix
from utils import load_json, load_pkl

courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

ModuleNotFoundError: No module named 'nltk'

## Exercise 4.1: Pre-processing

In [3]:
import re
from nltk.stem import WordNetLemmatizer
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

def preprocess(text):
    """
    Preprocesses a given text string into a list of lemmatized, clean tokens.

    Steps performed :
    1. Converts the text to lowercase.
    2. Removes all punctuation and non-alphanumeric characters (retaining spaces).
    3. Tokenizes the cleaned text into individual words using NLTK's word_tokenize.
    4. Filters out stopwords and single-character tokens.
    5. Lemmatizes each token to its base form (using WordNet Lemmatizer).

    Args :
        text (str) : The raw input text string to preprocess.

    Returns :
        List[str] : A list of processed, lemmatized tokens from the input text.
    """
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and single-char tokens
    tokens = [t for t in tokens if t not in stopwords and len(t) > 1]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

# Build document-term lists for all courses
doc_tokens = {}
vocab_counter = Counter()
for course in courses:
    cid = course['courseId']
    desc = course['description']
    tokens = preprocess(desc)
    doc_tokens[cid] = tokens
    vocab_counter.update(tokens)

# Filter out very frequent/infrequent terms
min_df = 2  # term must appear in at least 2 documents
max_df_ratio = 0.8  # term appears in less than 80% of docs
N = len(courses)
filtered_vocab = {term for term, df in vocab_counter.items()
                  if df >= min_df and df <= max_df_ratio * N}

# Prune tokens to filtered vocab
for cid, tokens in doc_tokens.items():
    doc_tokens[cid] = [t for t in tokens if t in filtered_vocab]

# Save preprocessed content to a new file
with open('preprocessed_courses.txt', 'w') as f:
    for cid, tokens in doc_tokens.items():
        # Write courseId and tokens as a JSON dictionary
        course_data = {'courseId': cid, 'tokens': tokens}
        f.write(json.dumps(course_data) + '\n')

# Print terms for COM-308 (Internet Analytics) in alphabetical order
target = 'COM-308'
terms_ix = sorted(set(doc_tokens[target]))
print("Preprocessed terms for {}: {}".format(target, terms_ix))

Preprocessed terms for COM-308: ['20', '30', '300', '50', 'acquired', 'activity', 'ad', 'advertisement', 'algebra', 'algorithm', 'analytics', 'application', 'assessment', 'auction', 'balance', 'based', 'cathedra', 'chain', 'class', 'cloud', 'clustering', 'collection', 'combination', 'commerce', 'communication', 'community', 'computing', 'concept', 'concrete', 'contained', 'coverage', 'current', 'data', 'datasets', 'decade', 'dedicated', 'designed', 'detection', 'dimensionality', 'draw', 'effectiveness', 'efficiency', 'end', 'exam', 'expected', 'explore', 'explores', 'field', 'final', 'foundational', 'framework', 'function', 'fundamental', 'good', 'graph', 'hadoop', 'hand', 'homework', 'important', 'information', 'infrastructure', 'inspired', 'internet', 'java', 'key', 'keywords', 'knowledge', 'lab', 'laboratory', 'large', 'linear', 'machine', 'main', 'map', 'markov', 'material', 'medium', 'midterm', 'mining', 'modeling', 'network', 'networking', 'number', 'online', 'outcome', 'past', '

## Exercise 4.1 Explanation : 

### We chose the following pre-processing steps :

1. **Lowercasing**  
   - **What :** Convert all text to lowercase.  
   - **Why :** Ensures that “Internet” and “internet” are treated identically, reducing duplicate entries in the vocabulary.

2. **Punctuation Removal**  
   - **What :** Strip out any character that is not a letter, digit, or whitespace using `re.sub(r"[^a-z0-9\s]", " ", text)`.  
   - **Why :** Punctuation (commas, periods, parentheses, etc.) generally does not carry semantic content in bag-of-words models and can be safely removed.

3. **Regex-Based Tokenization**  
   - **What :** Use `re.findall(r"\b[a-z0-9]+\b", text)` to extract contiguous alphanumeric tokens.  
   - **Why :** Avoids external dependencies (NLTK’s Punkt tokenizer) and is sufficient for splitting on whitespace and punctuation.

4. **Stopword Removal**  
   - **What :** Discard tokens found in `stopwords.pkl`.  
   - **Why :** Common English words (for example, “the”, “and”) appear in almost every document and add noise to similarity computations.

5. **Single‐Character Token Removal**  
   - **What :** Filter out any token of length ≤ 1.  
   - **Why :** Single letters (for example, “a”, “I”) are either stopwords or not informative.

6. **Lemmatization**  
   - **What :** Apply NLTK’s `WordNetLemmatizer` to each token.  
   - **Why :** Reduces inflected or derived forms to their dictionary root (for example, “computing” -> “compute”), consolidating features.

7. **Document-Frequency Filtering**  
   - **What :** Keep only terms that occur in at least `min_df=2` documents and in at most `max_df_ratio=0.8` of all documents.  
   - **Why :**  
     - Removing **infrequent** words (df < 2) reduces noise from typos or overly specialized terms.  
     - Removing **very frequent** words (df / N > 0.8) filters out quasi-stopwords that escaped the stopword list.

## Exercise 4.2: Term-document matrix

In [4]:
import numpy as np
from collections import Counter
from scipy.sparse import csr_matrix

# Build term <-> index and doc <-> index mappings
terms = sorted(filtered_vocab)                   # list of all retained vocabulary terms
term2idx = {t: i for i, t in enumerate(terms)}   # map term -> row index
doc2idx  = {cid: j for j, cid in enumerate(doc_tokens)}  # map courseId -> col index

M = len(terms)   # number of rows (terms)
N = len(doc_tokens)  # number of columns (documents)

# Compute document frequencies df[t]
df = Counter()
for tokens in doc_tokens.values():
    for t in set(tokens):
        df[t] += 1

# Compute idf[t] = log(N / df[t])
idf = {t: np.log(N / df[t]) for t in terms}

# Assemble sparse TF-IDF matrix X of shape (M, N) 
# We use max-normalized TF : tf_{t,d} = count(t in d) / max_t' count(t' in d)
rows, cols, data = [], [], []
for cid, tokens in doc_tokens.items():
    j = doc2idx[cid]
    tf_counter = Counter(tokens)
    max_tf = max(tf_counter.values()) if tf_counter else 1
    for t, tf in tf_counter.items():
        i = term2idx[t]
        tf_norm = tf / max_tf
        rows.append(i)
        cols.append(j)
        data.append(tf_norm * idf[t])

X = csr_matrix((data, (rows, cols)), shape=(M, N))

# Extract and print top-15 terms for COM-308 by TF-IDF score
target = 'COM-308'
j = doc2idx[target]
col = X[:, j].toarray().ravel()          # dense vector of TF-IDF scores for COM-308
top15_idx = np.argsort(col)[-15:][::-1]  # indices of the 15 largest scores

print(f"Top 15 terms in {target} by TF-IDF:")
for idx in top15_idx:
    term = terms[idx]
    score = col[idx]
    print(f"{term:20s}  {score:.4f}")

Top 15 terms in COM-308 by TF-IDF:
mining                3.1137
online                2.9099
social                2.6387
explore               2.4393
world                 2.3989
networking            2.2237
hadoop                2.0189
real                  1.9034
recommender           1.8838
service               1.8072
commerce              1.7879
auction               1.7879
datasets              1.6527
retrieval             1.6527
ad                    1.6013


In [5]:
# Added for part 2
from utils import save_pkl

save_pkl(X, 'X.pkl')
save_pkl(terms, 'terms.pkl')
doc_ids = list(doc_tokens.keys())
save_pkl(doc_ids, 'doc_ids.pkl')

## Exercise 4.2 Explanation : 

### Description :
In this section, we compute the **TF-IDF matrix** from scratch for the pre-processed corpus of course descriptions. The matrix represents each document as a vector of term scores, enabling document similarity analysis later on.

1. **Building Mappings**  
   - **terms :** List of unique terms in the filtered vocabulary.
   - **term2idx :** Maps each term to its row index in the matrix.  
   - **doc2idx :** Maps each document (by `courseId`) to its column index in the matrix.

2. **Calculating Document Frequencies (`df`)**  
   - For each term, count in how many documents it appears. This gives the denominator for the IDF calculation.

3. **Computing IDF (`idf`)**  
   - For each term, compute the Inverse Document Frequency as :  
      $[
     \text{idf}_t = \log\frac{N}{\text{df}_t}
     $]
     where \( N \) is the total number of documents. Terms appearing in many documents will have a low IDF, while rare terms will have a high IDF.

4. **Assembling the Sparse TF-IDF Matrix (`X`)**  
   - For each document :  
     - Count term frequencies (TF).  
     - Normalize each term frequency by the document’s maximum term frequency.  
     - Compute TF-IDF weight as $( \text{tf}_{t,d} \times \text{idf}_t $).  
     - Store non-zero entries as `(row, col, value)` for sparse construction.

5. **Extracting Top Terms for COM-308 (Internet Analytics)**  
   - Extract the column corresponding to COM-308.  
   - Sort by descending TF-IDF value.  
   - Print the top 15 terms along with their scores.




### Answer to Explain where the difference between the large scores and the small ones comes from.
The **difference** in scores is driven by two main factors :

- **Term Frequency (TF) :**  
  Terms like “mining”, “online”, and “social” appear frequently within the COM-308 description, increasing their TF weight.  
- **Inverse Document Frequency (IDF) :**  
  These terms are not common across the entire corpus of courses (low document frequency), which increases their IDF weight.  
Thus, the **highest scores** correspond to terms that are both important in the COM-308 document and rare elsewhere, reflecting their specificity and relevance.  

On the other hand :
- **Low scores** (though not shown here) would correspond to terms that either appear rarely in COM-308 (low TF) or are very common across many courses (low IDF), or both. These terms contribute little to distinguishing COM-308 from other documents.

This pattern is exactly what we expect in a **Vector Space Model** : unique, frequently used terms within a document (that aren’t common in the corpus) are key discriminators and receive higher weights.

## Exercise 4.3: Document similarity search

In [6]:
import numpy as np
from numpy.linalg import norm

def make_query_vector(query, term2idx, idf, preprocess, M):
    """
    Build a TF-IDF vector for an arbitrary query string.
    - query: raw text, e.g. "markov chains"
    - term2idx: mapping term -> row index in X
    - idf: dict term -> idf value
    - preprocess: function(text) -> list of tokens
    - M: size of vocabulary
    """
    tokens = preprocess(query)
    vec = np.zeros(M)
    tf_q = Counter(tokens)
    max_tf = max(tf_q.values()) if tf_q else 1
    for t, tf in tf_q.items():
        if t in term2idx:
            i = term2idx[t]
            vec[i] = (tf / max_tf) * idf.get(t, 0.0)
    return vec

def top_k_similar(query, X, courses, term2idx, idf, preprocess, doc2idx, k=5):
    """
    Compute cosine similarity between query and each document.
    Return top-k as list of (courseId, courseName, score).
    """
    M, N = X.shape
    qv = make_query_vector(query, term2idx, idf, preprocess, M)
    q_norm = norm(qv)
    results = []
    for course in courses:
        cid = course['courseId']
        j = doc2idx[cid]
        dv = X[:, j].toarray().ravel()
        d_norm = norm(dv)
        if q_norm > 0 and d_norm > 0:
            score = (qv @ dv) / (q_norm * d_norm)
        else:
            score = 0.0
        results.append((cid, course['name'], score))
    # sort descending by score
    results.sort(key=lambda x: x[2], reverse=True)
    return results[:k]

# Run searches and print
for query in ["markov chains", "facebook"]:
    print(f"\nTop 5 courses for query: \"{query}\"")
    for cid, name, score in top_k_similar(query, X, courses, term2idx, idf, preprocess, doc2idx, k=5):
        print(f"{cid:10s}  {name:45s}  score={score:.4f}")



Top 5 courses for query: "markov chains"
MGT-484     Applied probability & stochastic processes     score=0.5701
MATH-332    Applied stochastic processes                   score=0.5331
COM-516     Markov chains and algorithmic applications     score=0.4491
MGT-526     Supply chain management                        score=0.3694
MGT-602     Mathematical models in supply chain management  score=0.2955

Top 5 courses for query: "facebook"
EE-727      Computational Social Media                     score=0.1836
MSE-440     Composites technology                          score=0.0000
BIO-695     Image Processing for Life Science              score=0.0000
FIN-523     Global business environment                    score=0.0000
MICRO-614   Electrochemical nano-bio-sensing and bio/CMOS interfaces  score=0.0000


## Exercise 4.3 Explanation : 

### Description :
In this exercise, we are tasked with performing **document similarity search** using a **vector space model**. Specifically, we :  
1. Construct a **query vector** for each search term (“markov chains” and “facebook”) using the same TF-IDF weighting as used for the documents.  
2. Compute **cosine similarity** between the query vector and every document vector from the TF-IDF matrix \( X \) :  
   $$
   \text{sim}(d_i, d_j) = \frac{d_i^T d_j}{\|d_i\| \|d_j\|}
   $$
3. Rank the documents based on their similarity scores, and output the top 5 results for each query.

### Intuition and Interpretation of the Results :
- **For the query “markov chains”** :  
  The top results consist of courses directly related to stochastic processes and Markov chains (like, “Applied probability & stochastic processes” and “Markov chains and algorithmic applications”). This is expected because these courses contain the query terms or closely related terminology. The cosine similarity scores reflect the overlap between the query vector and the course vectors in the TF-IDF space.

- **For the query “facebook”** :  
  The top result is “Computational Social Media” with a low similarity score (0.1836), which makes sense because it likely mentions Facebook explicitly or discusses related topics like social networks. The remaining courses have a **similarity score of 0.0000**, indicating that they do not contain the term “facebook” (or any similar token) in their descriptions.  
  This occurs because the **query vector** only has nonzero values for terms like “facebook”, and if a document vector lacks these terms entirely, the dot product with the query vector is zero, leading to a cosine similarity of zero.

- **Why does this happen?**  
  Cosine similarity measures how well the directions of two vectors align. If a document contains none of the terms in the query, the vectors are orthogonal (dot product = 0), resulting in a similarity of 0. In contrast, courses with relevant content (like “markov chains”) have vectors with overlapping terms and thus higher similarity scores.

- **Possible improvements** :  
  - Incorporating **synonyms or semantically related terms** could improve retrieval. For example, “social network” might be relevant for “facebook”.  
  - Using **word embeddings (like, Word2Vec)** can capture semantic relationships even if exact terms don’t overlap.

### Conclusion :
This experiment demonstrates how a **vector space model with TF-IDF** can be used to perform document similarity searches. The method effectively highlights courses closely related to the query terms but struggles when no direct term overlap exists.
